In [ ]:
import random
import gc
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pickle
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import directed_hausdorff
from scipy.optimize import linear_sum_assignment
%matplotlib inline

sys.path.insert(0, '../../../methods')
from proposed import *

In [ ]:
# random seed
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
def get_coarray(X, sensor_pos, M):
    X_out = np.zeros((len(X), M-1, 2))
    X_mask = np.ones((len(X), M-1))

    for n in range(len(X)):
        #print(f"{n}/{len(X)}")
        counter = np.zeros(M-1)
        for i in range(sensor_pos[n].shape[0]):
            for j in range(i+1,sensor_pos[n].shape[0]):
                idx = int(abs(2*(sensor_pos[n][i]-sensor_pos[n][j]))-1)
                if sensor_pos[n][j] > sensor_pos[n][i]:
                    X_out[n,idx,0] = X_out[n,idx,0] + np.real(X[n][i,j])
                    X_out[n,idx,1] = X_out[n,idx,1] + np.imag(X[n][i,j])
                else:
                    X_out[n,idx,0] = X_out[n,idx,0] + np.real(X[n][j,i])
                    X_out[n,idx,1] = X_out[n,idx,1] + np.imag(X[n][j,i])
                X_mask[n,idx] = 0
                counter[idx] += 1

        for i in range(len(counter)):
            if counter[i] > 0:
                X_out[n,i,:] = X_out[n,i,:] / counter[i]

    return X_out, X_mask

In [ ]:
coarray = np.load('../../../data\experiment_1/scenario_1/coarray.npy')
coarray_mask = np.load('../../../data\experiment_1/scenario_1/coarray_mask.npy')
with open("../../../data\experiment_1/scenario_1/truths", "rb") as fp:   # Unpickling
    truths = pickle.load(fp)

In [ ]:
# manipulate labels for classification task
res = 1
low_lim = 30

labels = truths.copy()
for i in range(len(truths)):
    labels[i] = np.round((truths[i]-low_lim)*(1/res))

labels_ohe = np.zeros((len(labels),121))
for i in range(len(labels)):
    labels_ohe[i,labels[i].astype(int)] = 1
labels = labels_ohe

In [ ]:
# select channels
X = coarray
X_mask = coarray_mask
y = labels

# # standardize
# for i in range(X.shape[2]):
#     X[:,:,i] -= np.mean(X[:,:,i])
#     X[:,:,i] /= np.std(X[:,:,i])
#
# # nan to zero for diagonals
# data = np.nan_to_num(data)

In [ ]:
del labels
gc.collect()

In [ ]:
# train, val split
val_ratio = 0.1
X_train, X_val, y_train, y_val, m_train, m_val = train_test_split(X, y, X_mask, test_size=val_ratio, random_state=seed)

In [ ]:
# dataloader
class FastTensorDataLoader:
    """
    A DataLoader-like object for a set of tensors that can be much faster than
    TensorDataset + DataLoader because dataloader grabs individual indices of
    the dataset and calls cat (slow).
    Source: https://discuss.pytorch.org/t/dataloader-much-slower-than-manual-batching/27014/6
    """
    def __init__(self, *tensors, batch_size=32, shuffle=False):
        """
        Initialize a FastTensorDataLoader.
        :param *tensors: tensors to store. Must have the same length @ dim 0.
        :param batch_size: batch size to load.
        :param shuffle: if True, shuffle the data *in-place* whenever an
            iterator is created out of this object.
        :returns: A FastTensorDataLoader.
        """
        assert all(t.shape[0] == tensors[0].shape[0] for t in tensors)
        self.tensors = tensors

        self.dataset_len = self.tensors[0].shape[0]
        self.batch_size = batch_size
        self.shuffle = shuffle

        # Calculate # batches
        n_batches, remainder = divmod(self.dataset_len, self.batch_size)
        if remainder > 0:
            n_batches += 1
        self.n_batches = n_batches
    def __iter__(self):
        if self.shuffle:
            r = torch.randperm(self.dataset_len)
            self.tensors = [t[r] for t in self.tensors]
        self.i = 0
        return self

    def __next__(self):
        if self.i >= self.dataset_len:
            raise StopIteration
        batch = tuple(t[self.i:self.i+self.batch_size] for t in self.tensors)
        self.i += self.batch_size
        return batch

    def __len__(self):
        return self.n_batches

In [ ]:
# dataloader, model, criterion and optimizer initialization
batch_size = 512
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

train_loader = FastTensorDataLoader(X_train, y_train, m_train, batch_size=batch_size, shuffle=True)
val_loader = FastTensorDataLoader(X_val, y_val, m_val, batch_size=batch_size)
model1 = Covariance_Reconstructer().to(device)
model1.load_state_dict(torch.load('../../../results\experiment_1/scenario_1/proposed_step1.pt')) # map_location=torch.device('cpu')
model2 = DOA_Estimator().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=model2.parameters(), lr=learning_rate)

for param in model1.parameters():
    param.requires_grad = False

In [ ]:
def get_peaks(spectrum, n, device):
    spectrum = spectrum[0,:]
    spectrum_temp = torch.cat([torch.tensor([-1000]).to(device), spectrum, torch.tensor([-1000]).to(device)])

    crossings = ((torch.diff(torch.sign(torch.diff(spectrum_temp))) == -2)*1)
    crossings = torch.where(crossings==1)[0]
    crossing_spectrum = spectrum[crossings]
    crossings_sub = torch.argsort(crossing_spectrum, descending=True, dim=0)
    peaks = crossings[crossings_sub][:n]

    return peaks

In [ ]:
del X_train
del X_val
del y_train
del y_val
gc.collect()

In [ ]:
# training
num_epochs = 52
best_loss = 1000
lr_decay_patience = 10
lr_decay_counter = 0
train_loss_list = np.zeros(num_epochs)
train_rmse_list = np.zeros(num_epochs)
train_acc_list = np.zeros(num_epochs)
val_loss_list = np.zeros(num_epochs)
val_rmse_list = np.zeros(num_epochs)
val_acc_list = np.zeros(num_epochs)
# train_loss_list = np.load('/content/drive/MyDrive/Research/Experiments/train_loss.npy')
# train_rmse_list = np.load('/content/drive/MyDrive/Research/Experiments/train_rmse.npy')
# train_acc_list = np.load('/content/drive/MyDrive/Research/Experiments/train_acc.npy')
# val_loss_list = np.load('/content/drive/MyDrive/Research/Experiments/val_loss.npy')
# val_rmse_list = np.load('/content/drive/MyDrive/Research/Experiments/val_rmse.npy')
# val_acc_list = np.load('/content/drive/MyDrive/Research/Experiments/val_acc.npy')

for epoch in range(0,num_epochs):
    model1.eval()
    model2.train()
    running_loss = 0
    counter = 0
    p_list = []
    y_list = []
    t_list = []

    for batch_id, (X, y, m) in tqdm(enumerate(train_loader), leave=False, desc="training"):
        X = torch.tensor(X).float().to(device)
        y = torch.tensor(y).float().to(device)
        # t = torch.tensor(t).float().to(device)
        m = torch.tensor(m).type(torch.bool).to(device)

        X = model1(X,m)
        X = torch.cat([X, torch.cos(torch.angle(torch.complex(X[:,:,0], X[:,:,1]))).unsqueeze(2)], dim=2)
        X = torch.cat([X, torch.sin(torch.angle(torch.complex(X[:,:,0], X[:,:,1]))).unsqueeze(2)], dim=2)
        _, prob = model2(X,m)
        
        loss = criterion(prob,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.detach().data * y.shape[0]
        counter += y.shape[0]


        del X
        del y
        # del p
        del loss
        del prob
    gc.collect()

    train_loss = running_loss/counter
    train_loss_list[epoch] = train_loss

    del p_list
    del y_list
    del t_list
    gc.collect()

    torch.cuda.empty_cache()

    # validation
    with torch.no_grad():
        model1.eval()
        model2.eval()
        running_loss = 0
        counter = 0
        p_list = []
        y_list = []
        t_list = []

        for batch_id, (X, y, m) in tqdm(enumerate(val_loader), leave=False, desc="validation"):
            X = torch.tensor(X).float().to(device)
            y = torch.tensor(y).float().to(device)
            # t = torch.tensor(t).float().to(device)
            m = torch.tensor(m).type(torch.bool).to(device)

            X = model1(X, m)
            X = torch.cat([X, torch.cos(torch.angle(torch.complex(X[:,:,0], X[:,:,1]))).unsqueeze(2)], dim=2)
            X = torch.cat([X, torch.sin(torch.angle(torch.complex(X[:,:,0], X[:,:,1]))).unsqueeze(2)], dim=2)
            _, prob = model2(X, m)
            # p = get_peaks(prob,4)
            loss = criterion(prob,y)

            running_loss += loss.detach().data * y.shape[0]
            counter += y.shape[0]

            del X
            del y
            # del p
            del loss
            del prob
        gc.collect()

        val_loss = running_loss/counter
        val_loss_list[epoch] = val_loss

        del p_list
        del y_list
        del t_list
        gc.collect()

    print(f"epoch {epoch+1}, train-loss {train_loss:.4f}, val-loss {val_loss:.4f}, "
          f"train-rmse -, val-rmse -, train-acc -, val-acc -")

    if val_loss < best_loss:
        best_loss = val_loss
        lr_decay_counter = 0
        torch.save(model2.state_dict(), '../../../results\experiment_1/scenario_1/proposed_step2.pt')
        print(f"model is saved")
    else:
        lr_decay_counter += 1

    if lr_decay_counter == lr_decay_patience:
        lr_decay_counter = 0
        learning_rate *= 0.7
        learning_rate = max([learning_rate, 1e-7])
        print(f"learning rate decayed to {learning_rate:.4f}")

        for g in optimizer.param_groups:
            g['lr'] = learning_rate

    np.save('../../../results\experiment_1/scenario_1/train_loss_step2.npy', train_loss_list)
    np.save('../../../results\experiment_1/scenario_1/train_rmse_step2.npy', train_rmse_list)
    np.save('../../../results\experiment_1/scenario_1/train_acc_step2.npy', train_acc_list)
    np.save('../../../results\experiment_1/scenario_1/val_loss_step2.npy', val_loss_list)
    np.save('../../../results\experiment_1/scenario_1/val_rmse_step2.npy', val_rmse_list)
    np.save('../../../results\experiment_1/scenario_1/val_acc_step2.npy', val_acc_list)


In [ ]:
# loss curves
plt.figure(figsize=(10,5))
plt.plot(range(len(train_loss_list))[:52], train_loss_list[:52])
plt.plot(range(len(val_loss_list))[:52], val_loss_list[:52])
plt.title("training and validation loss curves")
plt.xlabel("epoch")
plt.ylabel("cross entropy loss")
plt.legend(["train", "val"])
plt.grid()

In [ ]:
snr_list = [-20,-15,-10,-5,0,5,10,15,20,25,30]
T_list = [100,200,500,1000,2000,5000,10000]
rmse_list1 = []
rmse_list2 = []
hausdorff_list1 = []
hausdorff_list2 = []
N_max = 4
M_max = 16

for snr in snr_list:
    # load data
    data = np.load(f'../../../data\experiment_1/scenario_1/data_test_snr{snr}_t1000.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    cm = [s['cm'] for s in data]
    sensor_pos = [s['sensor_pos'][:,0] for s in data]

    coarray, coarray_mask = get_coarray(cm, sensor_pos, M_max)

    # manipulate labels for classification task
    res = 1
    low_lim = 30
    labels = truths.copy()
    for i in range(len(truths)):
        labels[i] = np.round((truths[i]-low_lim)*(1/res))

    labels_ohe = np.zeros((len(labels),121))
    for i in range(len(labels)):
        labels_ohe[i,labels[i].astype(int)] = 1
    labels = labels_ohe

    # convert truths to matrix by padding
    truths_matrix = np.zeros((len(truths), N_max))
    num_sources = np.zeros(len(truths))
    for i in range(len(truths)):
        truths_matrix[i,0:truths[i].shape[0]] = truths[i].reshape(-1,)
        num_sources[i] = truths[i].shape[0]
    truths = truths_matrix

    # select channels
    X = coarray
    X_mask = coarray_mask
    y = labels

    # # standardize
    # for i in range(data.shape[2]):
    #     for j in range(data.shape[3]):
    #         for k in range(data.shape[1]):
    #             data[:,k,i,j] -= np.mean(data[:,k,i,j])
    #             data[:,k,i,j] /= np.std(data[:,k,i,j])
    #
    # # nan to zero for diagonals
    # data = np.nan_to_num(data)

    # dataloader, model, criterion and optimizer initialization
    batch_size = 1
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    test_loader = FastTensorDataLoader(X, y, truths, X_mask, num_sources, batch_size=batch_size)
    model1 = Covariance_Reconstructer().to(device)
    model1.load_state_dict(torch.load('../../../results\experiment_1/scenario_1/proposed_step1.pt',map_location=torch.device('cpu'))) # map_location=torch.device('cpu')
    model2 = DOA_Estimator().to(device)
    model2.load_state_dict(torch.load('../../../results\experiment_1/scenario_1/proposed_step2.pt',map_location=torch.device('cpu')))
    criterion = nn.BCEWithLogitsLoss()

    # testing
    with torch.no_grad():
        model1.eval()
        model2.eval()
        running_loss = 0
        counter = 0
        p_list = []
        t_list = []
        h_list = []
        r_list = []

        for batch_id, (X, y, t, m, n) in enumerate(test_loader):
            X = torch.tensor(X).float().to(device)
            y = torch.tensor(y).float().to(device)
            t = torch.tensor(t).float().to(device)
            m = torch.tensor(m).type(torch.bool).to(device)
            n = torch.tensor(n).long().to(device)

            X = model1(X,m)
            X = torch.cat([X, torch.cos(torch.angle(torch.complex(X[:,:,0], X[:,:,1]))).unsqueeze(2)], dim=2)
            X = torch.cat([X, torch.sin(torch.angle(torch.complex(X[:,:,0], X[:,:,1]))).unsqueeze(2)], dim=2)
            _, prob = model2(X,m)
            loss = criterion(prob,y)
            p = get_peaks(prob, n, device)
            p = p*res + low_lim
            t = t[0,:n] # for batch size of 1

            # calculate hausdorff distance
            hausdorff1 = directed_hausdorff(p.cpu().reshape(-1,1), t.cpu().reshape(-1,1))[0]
            hausdorff2 = directed_hausdorff(t.cpu().reshape(-1,1), p.cpu().reshape(-1,1))[0]
            h_list.append(max(hausdorff1, hausdorff2))

            # if cardinalities are equal, save for rmse calculation
            if t.shape[0] == p.shape[0]:
                t, _ = torch.sort(t)
                p, _ = torch.sort(p)
                r_list.append((t.detach().cpu().numpy() - p.detach().cpu().numpy())**2)

            if snr==10 and batch_id==0:
                np.save(f'../../../results\experiment_1/scenario_1/transformer_spectrum_{t.cpu().numpy()}deg.npy', F.softmax(prob[0,:], dim=0).cpu().numpy())

            running_loss += loss.item() * y.shape[0]
            counter += y.shape[0]
            p_list.append(p.detach().cpu().numpy())
            t_list.append(t.detach().cpu().numpy())

            del X
            del y
            del p
            del loss
            del prob
        gc.collect()

        test_loss = running_loss/counter
        test_hausdorff = np.mean(h_list)
        test_rmse = np.sqrt(np.mean(np.concatenate(r_list)))
        rmse_list1.append(test_rmse)
        hausdorff_list1.append(test_hausdorff)

        # del y_list
        gc.collect()

    print(f"snr {snr}dB, T {1000}, test-loss {test_loss:.4f}, test-rmse {test_rmse:.4f}, test-hausdorff: {test_hausdorff:.4f}")

    with open(f'../../../results\experiment_1/scenario_1/transformer_preds_snr{snr}_t1000', "wb") as fp:
        pickle.dump(p_list, fp)
    with open(f'../../../results\experiment_1/scenario_1/transformer_truths_snr{snr}_t1000', "wb") as fp:
        pickle.dump(t_list, fp)

# ------------------------------------------------------------------------------
for T in T_list:
    # load data
    data = np.load(f'../../../data\experiment_1/scenario_1/data_test_snr-10_t{T}.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    cm = [s['cm'] for s in data]
    sensor_pos = [s['sensor_pos'][:,0] for s in data]

    coarray, coarray_mask = get_coarray(cm, sensor_pos, M_max)

    # manipulate labels for classification task
    res = 1
    low_lim = 30
    labels = truths.copy()
    for i in range(len(truths)):
        labels[i] = np.round((truths[i]-low_lim)*(1/res))

    labels_ohe = np.zeros((len(labels),121))
    for i in range(len(labels)):
        labels_ohe[i,labels[i].astype(int)] = 1
    labels = labels_ohe

    # convert truths to matrix by padding
    truths_matrix = np.zeros((len(truths), N_max))
    num_sources = np.zeros(len(truths))
    for i in range(len(truths)):
        truths_matrix[i,0:truths[i].shape[0]] = truths[i].reshape(-1,)
        num_sources[i] = truths[i].shape[0]
    truths = truths_matrix

    # select channels
    X = coarray
    X_mask = coarray_mask
    y = labels

    # # standardize
    # for i in range(data.shape[2]):
    #     for j in range(data.shape[3]):
    #         for k in range(data.shape[1]):
    #             data[:,k,i,j] -= np.mean(data[:,k,i,j])
    #             data[:,k,i,j] /= np.std(data[:,k,i,j])
    #
    # # nan to zero for diagonals
    # data = np.nan_to_num(data)

    # dataloader, model, criterion and optimizer initialization
    batch_size = 1
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    test_loader = FastTensorDataLoader(X, y, truths, X_mask, num_sources, batch_size=batch_size)
    model1 = Covariance_Reconstructer().to(device)
    model1.load_state_dict(torch.load('../../../results\experiment_1/scenario_1/proposed_step1.pt', map_location=torch.device('cpu'))) # map_location=torch.device('cpu')
    model2 = DOA_Estimator().to(device)
    model2.load_state_dict(torch.load('../../../results\experiment_1/scenario_1/proposed_step2.pt', map_location=torch.device('cpu')))
    criterion = nn.CrossEntropyLoss()

    # testing
    with torch.no_grad():
        model1.eval()
        model2.eval()
        running_loss = 0
        counter = 0
        p_list = []
        t_list = []
        h_list = []
        r_list = []

        for batch_id, (X, y, t, m, n) in enumerate(test_loader):
            X = torch.tensor(X).float().to(device)
            y = torch.tensor(y).float().to(device)
            t = torch.tensor(t).float().to(device)
            m = torch.tensor(m).type(torch.bool).to(device)
            n = torch.tensor(n).long().to(device)

            X = model1(X,m)
            X = torch.cat([X, torch.cos(torch.angle(torch.complex(X[:,:,0], X[:,:,1]))).unsqueeze(2)], dim=2)
            X = torch.cat([X, torch.sin(torch.angle(torch.complex(X[:,:,0], X[:,:,1]))).unsqueeze(2)], dim=2)
            _, prob = model2(X,m)
            loss = criterion(prob,y)
            p = get_peaks(prob, n, device)
            p = p*res + low_lim
            t = t[0,:n] # for batch size of 1

            # calculate hausdorff distance
            hausdorff1 = directed_hausdorff(p.cpu().reshape(-1,1), t.cpu().reshape(-1,1))[0]
            hausdorff2 = directed_hausdorff(t.cpu().reshape(-1,1), p.cpu().reshape(-1,1))[0]
            h_list.append(max(hausdorff1, hausdorff2))

            # if cardinalities are equal, save for rmse calculation
            if t.shape[0] == p.shape[0]:
                t, _ = torch.sort(t)
                p, _ = torch.sort(p)
                r_list.append((t.detach().cpu().numpy() - p.detach().cpu().numpy())**2)

            running_loss += loss.item() * y.shape[0]
            counter += y.shape[0]
            p_list.append(p.detach().cpu().numpy())
            t_list.append(t.detach().cpu().numpy())

            del X
            del y
            del p
            del loss
            del prob
        gc.collect()

        test_loss = running_loss/counter
        test_hausdorff = np.mean(h_list)
        test_rmse = np.sqrt(np.mean(np.concatenate(r_list)))
        rmse_list2.append(test_rmse)
        hausdorff_list2.append(test_hausdorff)

        # del y_list
        gc.collect()

    print(f"snr -10dB, T {T}, test-loss {test_loss:.4f}, test-rmse {test_rmse:.4f}, test-hausdorff: {test_hausdorff:.4f}")

    with open(f'../../../results\experiment_1/scenario_1/transformer_preds_snr-10_t{T}', "wb") as fp:
        pickle.dump(p_list, fp)
    with open(f'../../../results\experiment_1/scenario_1/transformer_truths_snr-10_t{T}', "wb") as fp:
        pickle.dump(t_list, fp)

np.save('../../../results\experiment_1/scenario_1/transformer_rmse1.npy', rmse_list1)
np.save('../../../results\experiment_1/scenario_1/transformer_rmse2.npy', rmse_list2)
np.save('../../../results\experiment_1/scenario_1/transformer_hausdorff1.npy', hausdorff_list1)
np.save('../../../results\experiment_1/scenario_1/transformer_hausdorff2.npy', hausdorff_list2)

In [ ]:
# plot rmse values
plt.figure()
plt.plot(snr_list, rmse_list1, '-o')
plt.title("rmse values for different snr levels")
plt.xlabel("snr (dB)")
plt.ylabel("rmse (deg)")
plt.legend(['Dual Transformer'])
plt.yscale("log")
plt.grid()

plt.figure()
plt.plot(T_list, rmse_list2, '-o')
plt.title("rmse values for different snapshot numbers")
plt.xlabel("T")
plt.ylabel("rmse (deg)")
plt.legend(['Dual Transformer'])
plt.yscale("log")
plt.grid()

In [ ]:
# load data
data = np.load(f'../../../data\experiment_1/scenario_1/data_test_snr30_t1000.npy', allow_pickle=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model1 = Covariance_Reconstructer().to(device)
model1.load_state_dict(torch.load('../../../results\experiment_1/scenario_1/proposed_step1.pt', map_location=torch.device('cpu')))
model2 = DOA_Estimator().to(device)
model2.load_state_dict(torch.load('../../../results\experiment_1/scenario_1/proposed_step2.pt', map_location=torch.device('cpu')))

# get labels and data seperately
truths = [s['label'] for s in data]
cm = [s['cm'] for s in data]
cm_true = [s['cm_true'] for s in data]
sensor_pos = [s['sensor_pos'][:,0] for s in data]

M = 16
coarray, coarray_mask = get_coarray(cm, sensor_pos, M)
#coarray, coarray_mask = get_coarray(cm_true, sensor_pos, M)

# select channels
X = torch.tensor(coarray).float().to(device)
X_mask = torch.tensor(coarray_mask).type(torch.bool).to(device)

i=np.random.randint(low=0, high=len(X))

### both model
X = model1(X[i,:].unsqueeze(0), X_mask[i,:].unsqueeze(0))
X = torch.cat([X, torch.cos(torch.angle(torch.complex(X[:,:,0], X[:,:,1]))).unsqueeze(2)], dim=2)
X = torch.cat([X, torch.sin(torch.angle(torch.complex(X[:,:,0], X[:,:,1]))).unsqueeze(2)], dim=2)
_, prob = model2(X, X_mask[i,:].unsqueeze(0))
###

### only model 2
# X = X[i,:].unsqueeze(0)
# X = torch.cat([X, torch.cos(torch.angle(torch.complex(X[:,:,0], X[:,:,1]))).unsqueeze(2)], dim=2)
# X = torch.cat([X, torch.sin(torch.angle(torch.complex(X[:,:,0], X[:,:,1]))).unsqueeze(2)], dim=2)
# _, prob = model2(X, X_mask[i,:].unsqueeze(0))
###

spectrum = F.sigmoid(prob[0,:]).detach().cpu().numpy()
spectrum = spectrum/np.max(spectrum)
truth = truths[i]

plt.plot(np.arange(30,151), 10*np.log10(spectrum+1e-18), '-o')
#plt.plot(np.arange(30,151), spectrum, '-o')
for t in truth:
    plt.axvline(x=t, color = 'r', linestyle = '--')
plt.title(f"Sensor Positions: {sensor_pos[i]}")
plt.grid()
plt.show()